---
# **1 - Entscheidungsbäume**
---

In [ ]:
examples = [
  { "Alter": ">= 35", "Einkommen": "hoch",    "Bildung": "Abitur",    "Kandidat": "O" },
  { "Alter": "< 35",  "Einkommen": "niedrig", "Bildung": "Master",    "Kandidat": "O" },
  { "Alter": ">= 35", "Einkommen": "hoch",    "Bildung": "Bachelor",  "Kandidat": "M" },
  { "Alter": ">= 35", "Einkommen": "niedrig", "Bildung": "Abitur",    "Kandidat": "M" },
  { "Alter": ">= 35", "Einkommen": "hoch",    "Bildung": "Master",    "Kandidat": "O" },
  { "Alter": "< 35",  "Einkommen": "hoch",    "Bildung": "Bachelor",  "Kandidat": "O" },
  { "Alter": "< 35",  "Einkommen": "niedrig", "Bildung": "Abitur",    "Kandidat": "M" }
]


In [ ]:
# Knoten im DTL-Baum
class DTLNode:
  def classify(self, exmpl):
    raise NotImplementedError()


# Entscheidungs-Knoten
class DTLTree(DTLNode):
  def __init__(self, attr):
    self.attr = attr
    self.childs = dict()

  def classify(self, exmpl):
    attrVal = exmpl[self.attr]
    child = self.childs[attrVal]

    return child.classify(exmpl)


# Blatt, liefert Klasse
class DTLCLassLeaf(DTLNode):
  def __init__(self, val):
    self.value = val

  def classify(self, exmpl):
    return self.value

In [ ]:
def printTree(tree, lineStart=""):
  # Entscheidungs-Knoten
  if isinstance(tree, DTLTree):
      print(f"{lineStart}{tree.attr}")
      for key, child in tree.childs.items():
          print(f"{lineStart} ├─[{key}]")
          printTree(child, lineStart + " │   ")

  # Klassen-Blatt
  elif isinstance(tree, DTLCLassLeaf):
      print(f"{lineStart} └─[{tree.value}]")

  # Vereinigungsblatt (mehrdeutige Klassenzuordnung)
  elif isinstance(tree, DTLUnionLeaf):
      print(f"{lineStart} └─[UnionLeaf: {tree.values}]")


---
## **1.1 - CAL2**
---

In [ ]:
# Vereinigungsklasse (für CAL2)
class DTLUnionLeaf(DTLNode):
  def __init__(self, parent=None):
    self.parent = parent
    self.values = dict()

  def incVal(self, val):
    if val in self.values:
      self.values[val] += 1
    else:
      self.values[val] = 1

  def count(self):
    return sum(self.values.values())

  def maxVal(self):
    sum = self.count()
    maxCount = max(self.values.values())

    # ggf mehrere max. Klassen liefern
    return [ (k, v/sum) for k, v in self.values.items() if v == maxCount ]

  def classify(self, exmpl):
    return self


# Vereinigungsklasse durch Blatt ersetzten
def unionToClassLeaf(unionLeaf, val):
  if unionLeaf.parent is None:
    return DTLCLassLeaf(val=val)

  parent = unionLeaf.parent

  for key, child in parent.childs.items():
    if child is unionLeaf:
      parent.childs[key] = DTLCLassLeaf(val=val)
      return None

  raise RuntimeError()


# Vereinigungsklasse durch Sub-DTL-Baum ersetzten
def unionToTree(unionLeaf, examples, exmpl, val):
  attrList = list(examples[0].keys())

  if unionLeaf.parent is None:
    attr = attrList[0]
  else:
    parentAttr = unionLeaf.parent.attr
    parentAttrNr = attrList.index(parentAttr)

    # bereits alle Attribut verwendet
    if ((parentAttrNr + 1) >= len(attrList) - 1):
      return False

    attr = attrList[parentAttrNr + 1]

  attrValues = set([ e[attr] for e in examples ])
  subTree = DTLTree(attr)

  for v in attrValues:
    child = DTLUnionLeaf(parent=subTree)
    subTree.childs[v] = child

  i = exmpl[attr]
  subTree.childs[i].incVal(val)

  if unionLeaf.parent is None:
    return subTree

  parent = unionLeaf.parent

  for key, child in parent.childs.items():
    if child is unionLeaf:
      parent.childs[key] = subTree
      return None


In [ ]:
def dtl(examples, s1, s2):
  key = list(examples[0].keys())[-1]
  tree = DTLUnionLeaf()

  while True:
    # ganzer Duchlauf korrekt / nur noch Klassen als Endknoten ?
    finish = True

    # Durchlauf mit allen Beispielen:
    for e in examples:
      result = tree.classify(e)

      # endet Pfad in Klasse?
      if not isinstance(result, DTLUnionLeaf):
        continue

      # sonst Vereinigungsklasse +1
      result.incVal(e[key])
      finish = False

      # Summe >= S1 ?
      if (result.count() < s1):
        continue

      maxVals = result.maxVal()

      # mehr als 1 Wert >= S2 ? -> weitermachen
      if (len(maxVals) != 1):
        continue

      val, frac = maxVals[0]

      # Wert >= S2 ? -> Endknoten mit Klasse
      if (frac >= s2):
        newNode = unionToClassLeaf(result, val)
      # sonst -> neuer Entscheidungs-Knoten
      else:
        newNode = unionToTree(result, examples, e, val)

        # alle Attribute aufgebraucht -> kein Ergebnis
        if newNode == False:
          return None
      # Wurzelknoten 'tree' wurde geändert
      if newNode is not None:
        tree = newNode

    # nur noch Klassen als Endknoten -> fertig
    if finish == True:
      return tree


In [ ]:
tree = dtl(examples, 4, 0.7)
printTree(tree)


Alter
 ├─[< 35]
 │    └─[O]
 ├─[>= 35]
 │   Einkommen
 │    ├─[hoch]
 │    │    └─[O]
 │    ├─[niedrig]
 │    │    └─[M]


---
## **1.2 - ID3**
---

In [ ]:
from math import log2


def plurality(examples, attr):
  vals = [ e[attr] for e in examples ]
  valCount = { v : vals.count(v) for v in set(vals) }

  maxCount = max(valCount.values())
  maxVals = [ k for k, v in valCount.items() if v == maxCount ]

  return maxVals


def entropie(examples, key):
  key_vals = [ e[key] for e in examples ]
  sum = 0

  for k in set(key_vals):
    p_k = key_vals.count(k) / len(key_vals)
    sum += p_k * log2(p_k)

  return -sum


def infoGewinn(examples, attr, key):
  h_s = entropie(examples, key)
  r = 0

  attr_vals = set( e[attr] for e in examples )

  for v in attr_vals:
    vals_v = [ e for e in examples if e[attr] == v ]
    p_v = len(vals_v) / len(examples)
    r += p_v * entropie(vals_v, key)

  return h_s - r


def argmaxInformation(examples, attribs, key):
  maxVal = float('-inf')
  maxAttr = None

  for attr in attribs:
    val = infoGewinn(examples, attr, key)

    if (val > maxVal):
      maxVal = val
      maxAttr = attr

  return maxAttr


def id3(examples, attribs, default=None):
  key = list(examples[0].keys())[-1]

  if len(examples) == 0:
    return DTLCLassLeaf(default)

  if len(set([ e[key] for e in examples ])) == 1:
    uniqueVal = examples[0][key]
    return DTLCLassLeaf(uniqueVal)

  if len(attribs) == 0:
    maxPlurality = plurality(examples, key)
    return DTLCLassLeaf(maxPlurality)

  attr = argmaxInformation(examples, attribs, key)
  tree = DTLTree(attr)

  nextDefault = plurality(examples, key)
  nextAttribs = [ a for a in attribs if a != attr ]

  for v in set( e[attr] for e in examples ):
    examples_v = [ e for e in examples if e[attr] == v ]

    subTree = id3(examples_v, nextAttribs, nextDefault)
    tree.childs[v] = subTree

  return tree


In [ ]:
attribs = [ e for e in examples[0].keys() if e != "Kandidat" ]

idTree = id3(examples, attribs)
printTree(idTree)


Bildung
 ├─[Abitur]
 │   Einkommen
 │    ├─[hoch]
 │    │    └─[O]
 │    ├─[niedrig]
 │    │    └─[M]
 ├─[Master]
 │    └─[O]
 ├─[Bachelor]
 │   Alter
 │    ├─[< 35]
 │    │    └─[O]
 │    ├─[>= 35]
 │    │    └─[M]


---
# **2 - Pruning**
---

**Ausgangssituation:**

$$
\alpha \ := \ x_3(x_2(x_1(C,A), x_1(B,A)), x_1(x_2(C,B), A))
\\ \\
$$

---

$$
\\
$$


Transformationsregel anwenden:
$$
x_2(x_1(C,A), x_1(B,A)) \ \Longleftrightarrow \
x_1(x_2(C,B), x_2(A,A))
\\
$$

$$
\alpha \ = \ x_3(x_1(x_2(C,B), x_2(A,A)), x_1(x_2(C,B), A))
\\ \\
$$


Pruning für bedingt relevante Attribute anwenden:
$$
x_2(A,A)
\ \Longleftrightarrow \
A
\\
$$

$$
\alpha \ = \ x_3(x_1(x_2(C,B), A), x_1(x_2(C,B), A))
\\ \\
$$


Pruning für bedingt relevante Attribute anwenden:
$$
x_3(x_1(x_2(C,B), A), x_1(x_2(C,B), A))
\ \Longleftrightarrow \
x_1(x_2(C,B), A)
\\
$$

$$
\alpha \ = \ x_1(x_2(C,B), A)
\\ \\
$$

---
# **3 - Weka**
---

---
## **3.1 - Training mit J48**
---


### **3.1.1 - zoo.csv**
---


#### **Ergebnisse**

```
.
|
feathers <= 0
|   |
|   milk <= 0
|   |   |
|   |   backbone <= 0
|   |   |   |
|   |   |   airbone <= 0
|   |   |   |   |
|   |   |   |   predator <= 0
|   |   |   |   |   |
|   |   |   |   |   legs <= 2:
|   |   |   |   |   |   --> shellfish (2.0)
|   |   |   |   |   |
|   |   |   |   |   legs > 2:
|   |   |   |   |       --> insect (2.0)
|   |   |   |   |
|   |   |   |   predator > 0:
|   |   |   |       --> shellfish (8.0)
|   |   |   |
|   |   |   airbone > 0:
|   |   |        --> insect (6.0)
|   |   |
|   |   backbone > 0
|   |      |
|   |      fins <= 0
|   |      |   |
|   |      |   tail <= 0:
|   |      |   |   --> amphibian (3.0)
|   |      |   |
|   |      |   tail > 0:
|   |      |       --> reptile (6.0/1.0)
|   |      |
|   |      fins > 0:
|   |          --> fish (13.0)
|   |
|   milk > 0:
|       --> mammal (41.0)
|
feathers > 0:
    --> bird (20.0)
```

---

#### **Accuracy**

**Training-Set:**

$99.01 \ \%$

$$
\\
$$

**k-Fold-Cross-Valdidation (k = 20):**

$92.08 \ \%$

---

#### **Confusion Matrix**

```
  a  b  c  d  e  f  g |  <-- classified as
 ---------------------|--------------------
 41  0  0  0  0  0  0 |  a = mammal
  0 13  0  0  0  0  0 |  b = fish
  0  0 20  0  0  0  0 |  c = bird
  0  0  0  8  2  0  0 |  d = shellfish
  0  0  0  3  5  0  0 |  e = insect
  0  0  0  0  0  3  1 |  f = amphibian
  0  1  0  0  1  0  3 |  g = reptile
```

für *Mammal*, *Fish* und *Bird* ergibt sich nahzu $100 \ \%$ Accuracy. Für *Insect* und *Shellfish* hingegen sind Recall und Precision schwach - es kommt  zu Verwechselungen (*Insect* wird als *Shellfish* klassifiziert und andersrum). *Amphibian* hat $100 \ \%$ Precision, aber niedrigen Recall (wird zT. als *Reptile* klassifiziert). Entsprechend sieht Precision bei *Reptile* aus, Recall ist hier noch schlechter (es kommt zu Fehl-Klassifikation als *Bird* oder *Fish*).

$$
\\
$$

---



### **3.1.2 - restaurant.csv**
---

#### **Ergebnisse**


```
.
|
Gaeste = Some:
|    --> Yes (4.0)
|
Gaeste =  Full:
|    --> Yes (6.0/2.0)
|
Gaeste = None:
     --> No (2.0)
```

---
#### **Accuracy**

**Training-Set:**

$83.33 \ \%$

$$
\\
$$

**k-Fold-Cross-Valdidation (k = 3):**

$66.67 \ \%$

---
#### **Confusion Matrix**

```
 a b | <-- classified as
-----|-------------------
 3 3 | a = Yes
 1 5 | b = No
```

Für **Warten =** *Yes* ist die Precision relativ hoch (im Vergleich zur Gesamt-Accuracy), Recall aber deutlich schlechter - *Yes* werden zu gleichen Teilen als *Yes* und *No* klassifiziert. Bei **Warten =** *No* ist dagegen Recall hoch, Precision niedrig - die meisten *Nos* werden korrekt klassifiziert, aber viele *Nos* sind eig *Yes*.   

Das Problem ist hier, dass der Algorithmus nur anhand von **Gäste** klassifiziert. Für *None* und *Some* ist das vollständig korrekt, aber auch *Full* wird als *Yes* klassifiziert, obwohl es hier noch viele *Nos* gibt. Daher ergeben sich die hohen False-Negatives bzw. der schwache Recall für *Yes* und schwache Precision für *No*.


---
## **3.2 - ARFF-Format**
---

**Ordinal / Numeric:**

Ganzzahlige oder Reele Werte

$$
\\
$$

**Nominal:**

Abzählbare diskrete Klassen (die erlaubten Werte / Klassen werden in der Datei spezifiziert)

$$
\\
$$

**String:**

Beliebige Zeichenketten (im Gegensatz zu den **Nominalen** Werten, die beschränkt sind auf die erlaubten Klassen)

$$
\\
$$

---

[zoo.arff](https://github.com/Cpp-dino/hsbi_ki_portfolio/blob/aad8885ded38edb70f0f48871cba54010d58fcc9/aufgaben/zoo.arff)


[restaurant.arff](https://github.com/Cpp-dino/hsbi_ki_portfolio/blob/aad8885ded38edb70f0f48871cba54010d58fcc9/aufgaben/restaurant.arff)

---
## **3.3 - Training mit ID3 und J48**
---

### **3.3.1 - zoo.csv**

---

#### **Ergebnisse**

```
.
|
legs = 0
|  |
|  fins = 0
|  |  toothed = 0:
|  |  |    --> shellfish
|  |  |
|  |  toothed = 1:
|  |       --> reptile
|  |
|  fins = 1
|     eggs = 0:
|     |    --> mammal
|     |
|     eggs = 1:
|          --> fish
|
legs = 2
|  |
|  hair = 0:
|  |    --> bird
|  |
|  hair = 1:
|       --> mammal
|
legs = 4
|  |
|  hair = 0
|  |  |
|  |  aquatic = 0:
|  |  |    --> reptile
|  |  |
|  |  aquatic = 1
|  |     |
|  |     toothed = 0:
|  |     |    --> shellfish
|  |     |
|  |     toothed = 1:
|  |          --> amphibian
|  |  
|  hair = 1: mammal
|       --> mammal
|
legs = 5: shellfish
|    --> shellfish
|
legs = 6
|  |
|  aquatic = 0: insect
|  |    --> insect
|  |  
|  aquatic = 1: shellfish
|       --> shellfish
|
legs = 8:
     --> shellfish
```

---
#### **Accuracy**

**Training-Set:**

$100 \ \%$

$$
\\
$$

**k-Fold-Cross-Valdidation (k = 10):**

$98.02 \ \%$

---

#### **Confusion Matrix**

```
  a  b  c  d  e  f  g |  <-- classified as
 ---------------------|--------------------
 41  0  0  0  0  0  0 |  a = mammal
  0 13  0  0  0  0  0 |  b = fish
  0  0 20  0  0  0  0 |  c = bird
  0  0  0  8  0  1  0 |  d = shellfish
  0  0  0  0  8  0  0 |  e = insect
  0  0  0  0  0  4  0 |  f = amphibian
  0  0  0  0  0  0  5 |  g = reptile

```

Accuracy ist nahzu für alle Klassen $100 \ \%$, nur bei *Amphibien* gibt es etwas niedrigere Precision bzw. bei *Shellfish* niedrigeren Recall (*Shellfish* wird zT. als *Amphibien* klassifiziert).

$$
\\
$$

---


### **3.3.2 - restaurant.csv**
---

#### **Ergebnisse**


```
.
|
Gaeste = Some:
|    --> Yes
|
Gaeste =  None:
|    --> No
|
Gaeste = Full:
   |
   Typ = French:
   |    --> No
   |
   Typ = Thai
   |  |
   |  Fr = Yes:
   |  |    --> Yes
   |  |
   |  Fr = No:
   |       --> No
   |
   Typ = Burger
   |  |
   |  Hunger = Yes:
   |  |    --> Yes
   |  |
   |  Hunger = No:
   |       --> No
   |
   Typ = Italian:
        --> No
```

---
#### **Accuracy**

**Training-Set:**

$100 \ \%$

$$
\\
$$

**Train-Test-Split (75% / 25%):**

$66.67 \ \%$

---
#### **Confusion Matrix**

```
 a b | <-- classified as
-----|-------------------
 1 0 | a = Yes
 1 1 | b = No
```

Für **Warten =** *No* ist die Precision nun bei $100 \ \%$, dafür aber nun hier Recall schlechter - *Nos* werden zu gleichen Teilen als *Yes* und *No* klassifiziert. Entsprechend ist bei **Warten =** *Yes* die Precision schlecht, Recall aber bei $100 \ \%$ - die meisten *Yes* werden korrekt klassifiziert, aber viele *Yes* sind eig *Nos*.

$$
\\
$$

---